In [1]:
import pandas as pd
!pip install imblearn

In [2]:
heart=pd.read_csv("/Users/ronak/Library/Containers/com.microsoft.Excel/Data/Downloads/CVD_cleaned 4.csv")


In [3]:
heart.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150,32.66,14.54,Yes,0,30,16,12
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165,77.11,28.29,No,0,30,0,4
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163,88.45,33.47,No,4,12,3,16
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180,93.44,28.73,No,0,30,30,8
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191,88.45,24.37,Yes,0,8,4,0


In [4]:
heart.shape

(308854, 19)

In [5]:
heart["Heart_Disease"].value_counts()

Heart_Disease
No     283883
Yes     24971
Name: count, dtype: int64

In [6]:
# Convering the column names into lower case and replacing the space with an underscore
heart.columns = heart.columns.str.lower().str.replace(" ", "_")

#Changing the name of a big column

heart.rename(columns = {'height_(cm)' : 'height', 'weight_(kg)' : 'weight', 'green_vegetables_consumption' : 'vegetables_consumption', 'friedpotato_consumption' : 'potato_consumption'}, inplace = True)

In [7]:
heart['checkup'] = heart['checkup'].replace('Within the past 2 years', 'Past 2 years')
heart['checkup'] = heart['checkup'].replace('Within the past year', 'Past 1 year')
heart['checkup'] = heart['checkup'].replace('Within the past 5 years', 'Past 5 years')
heart['checkup'] = heart['checkup'].replace('5 or more years ago', 'More than 5 years')


heart['diabetes'] = heart['diabetes'].replace('No, pre-diabetes or borderline diabetes', 'No Pre Diabetes')
heart['diabetes'] = heart['diabetes'].replace('Yes, but female told only during pregnancy', 'Only during pregnancy')

heart['age_category'] = heart['age_category'].replace('18-24', 'Young')
heart['age_category'] = heart['age_category'].replace('25-29', 'Adult')
heart['age_category'] = heart['age_category'].replace('30-34', 'Adult')
heart['age_category'] = heart['age_category'].replace('35-39', 'Adult')
heart['age_category'] = heart['age_category'].replace('40-44', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('45-49', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('50-54', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('55-59', 'Senior-Adult')
heart['age_category'] = heart['age_category'].replace('60-64', 'Senior-Adult')
heart['age_category'] = heart['age_category'].replace('65-69', 'Elderly')
heart['age_category'] = heart['age_category'].replace('70-74', 'Elderly')
heart['age_category'] = heart['age_category'].replace('75-79', 'Elderly')
heart['age_category'] = heart['age_category'].replace('80+', 'Elderly')

In [8]:
col = ['alcohol_consumption', 'fruit_consumption', 'vegetables_consumption', 'potato_consumption']

for i in col:
    heart[i] = heart[i].astype(int)

In [9]:
# Define BMI ranges and labels for each group
bmi_bins = [12.02, 18.3, 26.85, 31.58, 37.8, 100]
bmi_labels = ['Underweight', 'Normal weight', 'Overweight', 'Obese I', 'Obese II']
heart['bmi_group'] = pd.cut(heart['bmi'], bins=bmi_bins, labels=bmi_labels, right=False)

In [10]:
column_to_move = heart.pop('bmi_group')
heart.insert(14, 'bmi_group', column_to_move)

In [11]:
heart['bmi_group'] = heart['bmi_group'].astype('object')

In [12]:
# Import the OneHotEncoder class from scikit-learn

heart['heart_disease'] = heart['heart_disease'].map({'Yes':1, 'No':0})




In [13]:
# Save to a CSV file in the current notebook folder
heart.to_csv("processed_data.csv", index=False)


In [14]:
from IPython.display import FileLink

# Create a clickable download link
FileLink("processed_data.csv")


/Users/ronak/Desktop/ML Projects/processed_data.csv

In [13]:
# Separate target
y = heart["heart_disease"]   # target column
X = heart.drop(columns=["heart_disease"])   # features only

# Identify feature types
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Transformers
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

# ColumnTransformer (only applied on features, not target!)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", oh_transformer, cat_features),
    ]
)

# Transform features
X= preprocessor.fit_transform(X)


In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

In [15]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and testing sets for diabetes balanced

X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler_d = StandardScaler()
X_train_scaled = scaler_d.fit_transform(X_train)
X_test_scaled = scaler_d.transform(X_test)

In [17]:
# Create the models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
lr_d = LogisticRegression()
rf_d = RandomForestClassifier()

# Fit the models

lr_d.fit(X_train_scaled, y_train)
rf_d.fit(X_train_scaled, y_train)

# Make predictions

lr_pred_d = lr_d.predict(X_test_scaled)
rf_pred_d = rf_d.predict(X_test_scaled)

In [18]:
from sklearn.metrics import classification_report, roc_auc_score
lr_d_report = classification_report(y_test, lr_pred_d)

rf_d_report = classification_report(y_test, rf_pred_d)


print("="*40, "Logistic regression report:", "="*45, '\n')
print(lr_d_report)



print("="*40, "Random forest report:", "="*45, '\n')
print(rf_d_report)


======================================== Logistic regression report: ============================================= 

              precision    recall  f1-score   support

           0       0.79      0.73      0.76     85071
           1       0.75      0.81      0.78     85259

    accuracy                           0.77    170330
   macro avg       0.77      0.77      0.77    170330
weighted avg       0.77      0.77      0.77    170330

======================================== Random forest report: ============================================= 

              precision    recall  f1-score   support

           0       0.95      0.96      0.95     85071
           1       0.96      0.95      0.95     85259

    accuracy                           0.95    170330
   macro avg       0.95      0.95      0.95    170330
weighted avg       0.95      0.95      0.95    170330

